In [1]:
import numpy as np
import subprocess
import tqdm
import pandas as pd

import os
import os.path as osp

import glob

import h5py
import uproot

import torch
from torch import nn
from torch_geometric.data import Data
from torch_geometric.data import Dataset
from torch_geometric.data import DataLoader

import awkward as ak
import random

In [3]:
v_path = '/vols/cms/mm1221/Data/100k/5pi/train/raw/train.root'

In [4]:
import uproot
import matplotlib.pyplot as plt

# Open files
v_file = uproot.open(v_path)

# Inspect contents
print("Electron file contents:")
print(v_file.keys())


Electron file contents:
['tracksters;10', 'tracksters;9', 'clusters;31', 'clusters;30', 'candidates;1', 'trackstersMerged;11', 'trackstersMerged;10', 'associations;4', 'associations;3', 'simtrackstersSC;21', 'simtrackstersSC;20', 'simtrackstersCP;22', 'simtrackstersCP;21', 'tracks;1', 'simTICLCandidate;1']


In [6]:
ass = v_file['associations;4']['tsCLUE3D_recoToSim_CP'].array()

In [18]:
print(ass[2][])

[2, 3, 0, 1]


In [30]:
import glob
import os.path as osp
import uproot
import awkward as ak
import torch
import numpy as np
import random
import tqdm
from torch_geometric.data import Data, Dataset

import numpy as np
import subprocess
import tqdm
import pandas as pd

import os
import os.path as osp

import glob

import h5py
import uproot

import torch
from torch import nn
from torch_geometric.data import Data
from torch_geometric.data import Dataset
from torch_geometric.data import DataLoader

import awkward as ak
import random
def find_highest_branch(path, base_name):
    with uproot.open(path) as f:
        # Find keys that exactly match the base_name (not containing other variations)
        branches = [k for k in f.keys() if k.startswith(base_name + ';')]
        
        # Sort and select the highest-numbered branch
        sorted_branches = sorted(branches, key=lambda x: int(x.split(';')[-1]))
        return sorted_branches[-1] if sorted_branches else None

class CCV1(Dataset):
    r'''
    Loads trackster-level features and associations for positive/negative edge creation.
    '''

    url = '/dummy/'

    def __init__(self, root, transform=None, max_events=1e8, inp='train'):
        super(CCV1, self).__init__(root, transform)
        self.inp = inp
        self.max_events = max_events
        self.fill_data(max_events)

    def fill_data(self, max_events):
        counter = 0
        print("### Loading tracksters data")


        for path in tqdm.tqdm(self.raw_paths):
            print(path)
            
            tracksters_path = find_highest_branch(path, 'tracksters')
            associations_path = find_highest_branch(path, 'associations')
            simtrack = find_highest_branch(path, 'simtrackstersCP')
            # Load tracksters features in chunks
            for array in uproot.iterate(
                f"{path}:{tracksters_path}",
                [
                    "time", "raw_energy",
                    "barycenter_x", "barycenter_y", "barycenter_z", 
                    "barycenter_eta", "barycenter_phi",
                    "EV1", "EV2", "EV3",
                    "eVector0_x", "eVector0_y", "eVector0_z",
                    "sigmaPCA1", "sigmaPCA2", "sigmaPCA3"
                ],
            ):

                tmp_time = array["time"]
                tmp_raw_energy = array["raw_energy"]
                tmp_bx = array["barycenter_x"]
                tmp_by = array["barycenter_y"]
                tmp_bz = array["barycenter_z"]
                tmp_beta = array["barycenter_eta"]
                tmp_bphi = array["barycenter_phi"]
                tmp_EV1 = array["EV1"]
                tmp_EV2 = array["EV2"]
                tmp_EV3 = array["EV3"]
                tmp_eV0x = array["eVector0_x"]
                tmp_eV0y = array["eVector0_y"]
                tmp_eV0z = array["eVector0_z"]
                tmp_sigma1 = array["sigmaPCA1"]
                tmp_sigma2 = array["sigmaPCA2"]
                tmp_sigma3 = array["sigmaPCA3"]
                
                
                vert_array = []
                for vert_chunk in uproot.iterate(
                    f"{path}:{simtrack}",
                    ["barycenter_x"],
                ):
                    vert_array = vert_chunk["barycenter_x"]
                    break  # Since we have a matching chunk, no need to continue
                

                # Now load the associations for the same events/chunk
                # 'tsCLUE3D_recoToSim_CP' gives association arrays like [[1,0],[0,1],...]
                # Make sure we read from the same events
                tmp_array = []
                for assoc_chunk in uproot.iterate(
                    f"{path}:{associations_path}",
                    ["tsCLUE3D_recoToSim_CP"],
                ):
                    tmp_array = assoc_chunk["tsCLUE3D_recoToSim_CP"]
                    break  # Since we have a matching chunk, no need to continue
                
                
                skim_mask = []
                print(len(vert_array))
                for e in vert_array:
                    if 2 <= len(e) <= 5:
                        skim_mask.append(True)
                        print('true', len(e))
                    elif len(e) == 0:
                        skim_mask.append(False)
                        print('0', len(e))
                    else:
                        skim_mask.append(False)
                        print('false', len(e))


                tmp_time = tmp_time[skim_mask]
                tmp_raw_energy = tmp_raw_energy[skim_mask]
                tmp_bx = tmp_bx[skim_mask]
                tmp_by = tmp_by[skim_mask]
                tmp_bz = tmp_bz[skim_mask]
                tmp_beta = tmp_beta[skim_mask]
                tmp_bphi = tmp_bphi[skim_mask]
                tmp_EV1 = tmp_EV1[skim_mask]
                tmp_EV2 = tmp_EV2[skim_mask]
                tmp_EV3 = tmp_EV3[skim_mask]
                tmp_eV0x = tmp_eV0x[skim_mask]
                tmp_eV0y = tmp_eV0y[skim_mask]
                tmp_eV0z = tmp_eV0z[skim_mask]
                tmp_sigma1 = tmp_sigma1[skim_mask]
                tmp_sigma2 = tmp_sigma2[skim_mask]
                tmp_sigma3 = tmp_sigma3[skim_mask]
                tmp_array = tmp_array[skim_mask]
                
                skim_mask = []
                for e in tmp_array:
                    if 2 <= len(e) <= 5:
                        skim_mask.append(True)
                        print('true', len(e))
                    elif len(e) == 0:
                        skim_mask.append(False)
                        print('0', len(e))
                    else:
                        skim_mask.append(False)
                        print('false', len(e))
                        
                tmp_time = tmp_time[skim_mask]
                tmp_raw_energy = tmp_raw_energy[skim_mask]
                tmp_bx = tmp_bx[skim_mask]
                tmp_by = tmp_by[skim_mask]
                tmp_bz = tmp_bz[skim_mask]
                tmp_beta = tmp_beta[skim_mask]
                tmp_bphi = tmp_bphi[skim_mask]
                tmp_EV1 = tmp_EV1[skim_mask]
                tmp_EV2 = tmp_EV2[skim_mask]
                tmp_EV3 = tmp_EV3[skim_mask]
                tmp_eV0x = tmp_eV0x[skim_mask]
                tmp_eV0y = tmp_eV0y[skim_mask]
                tmp_eV0z = tmp_eV0z[skim_mask]
                tmp_sigma1 = tmp_sigma1[skim_mask]
                tmp_sigma2 = tmp_sigma2[skim_mask]
                tmp_sigma3 = tmp_sigma3[skim_mask]
                tmp_array = tmp_array[skim_mask]

                
                # Concatenate or initialize storage
                if counter == 0:
                    self.time = tmp_time
                    self.raw_energy = tmp_raw_energy
                    self.bx = tmp_bx
                    self.by = tmp_by
                    self.bz = tmp_bz
                    self.beta = tmp_beta
                    self.bphi = tmp_bphi
                    self.EV1 = tmp_EV1
                    self.EV2 = tmp_EV2
                    self.EV3 = tmp_EV3
                    self.eV0x = tmp_eV0x
                    self.eV0y = tmp_eV0y
                    self.eV0z = tmp_eV0z
                    self.sigma1 = tmp_sigma1
                    self.sigma2 = tmp_sigma2
                    self.sigma3 = tmp_sigma3
                    self.assoc = tmp_array
                else:
                    self.time = ak.concatenate((self.time, tmp_time))
                    self.raw_energy = ak.concatenate((self.raw_energy, tmp_raw_energy))
                    self.bx = ak.concatenate((self.bx, tmp_bx))
                    self.by = ak.concatenate((self.by, tmp_by))
                    self.bz = ak.concatenate((self.bz, tmp_bz))
                    self.beta = ak.concatenate((self.beta, tmp_beta))
                    self.bphi = ak.concatenate((self.bphi, tmp_bphi))
                    self.EV1 = ak.concatenate((self.EV1, tmp_EV1))
                    self.EV2 = ak.concatenate((self.EV2, tmp_EV2))
                    self.EV3 = ak.concatenate((self.EV3, tmp_EV3))
                    self.eV0x = ak.concatenate((self.eV0x, tmp_eV0x))
                    self.eV0y = ak.concatenate((self.eV0y, tmp_eV0y))
                    self.eV0z = ak.concatenate((self.eV0z, tmp_eV0z))
                    self.sigma1 = ak.concatenate((self.sigma1, tmp_sigma1))
                    self.sigma2 = ak.concatenate((self.sigma2, tmp_sigma2))
                    self.sigma3 = ak.concatenate((self.sigma3, tmp_sigma3))
                    self.assoc = ak.concatenate((self.assoc, tmp_array))

                counter += len(tmp_bx)
                if counter >= max_events:
                    print(f"Reached {max_events} events!")
                    break
            if counter >= max_events:
                break

    def download(self):
        raise RuntimeError(
            f'Dataset not found. Please download it from {self.url} and move all '
            f'*.root files to {self.raw_dir}')

    def len(self):
        return len(self.time)

    @property
    def raw_file_names(self):
        raw_files = sorted(glob.glob(osp.join(self.raw_dir, '*.root')))
        return raw_files

    @property
    def processed_file_names(self):
        return []



    def get(self, idx):
        
        def reconstruct_array(grouped_indices):
            # Find the maximum index to determine the array length
            max_index = max(max(indices) for indices in grouped_indices.values())

            # Initialize an array with the correct size, filled with a placeholder (e.g., -1)
            reconstructed = [-1] * (max_index + 1)

            # Populate the array based on the dictionary
            for value, indices in grouped_indices.items():
                for idx in indices:
                    reconstructed[idx] = value

            return reconstructed
        # Extract per-event arrays
        event_time = self.time[idx]
        event_raw_energy = self.raw_energy[idx]
        event_bx = self.bx[idx]
        event_by = self.by[idx]
        event_bz = self.bz[idx]
        event_beta = self.beta[idx]
        event_bphi = self.bphi[idx]
        event_EV1 = self.EV1[idx]
        event_EV2 = self.EV2[idx]
        event_EV3 = self.EV3[idx]
        event_eV0x = self.eV0x[idx]
        event_eV0y = self.eV0y[idx]
        event_eV0z = self.eV0z[idx]
        event_sigma1 = self.sigma1[idx]
        event_sigma2 = self.sigma2[idx]
        event_sigma3 = self.sigma3[idx]
        event_assoc = self.assoc[idx]  # Each is now an array (e.g., [0, 1, 2]) indicating the pion group

        # Convert each to numpy arrays if needed
        event_time = np.array(event_time)
        event_raw_energy = np.array(event_raw_energy)
        event_bx = np.array(event_bx)
        event_by = np.array(event_by)
        event_bz = np.array(event_bz)
        event_beta = np.array(event_beta)
        event_bphi = np.array(event_bphi)
        event_EV1 = np.array(event_EV1)
        event_EV2 = np.array(event_EV2)
        event_EV3 = np.array(event_EV3)
        event_eV0x = np.array(event_eV0x)
        event_eV0y = np.array(event_eV0y)
        event_eV0z = np.array(event_eV0z)
        event_sigma1 = np.array(event_sigma1)
        event_sigma2 = np.array(event_sigma2)
        event_sigma3 = np.array(event_sigma3)
        event_assoc = np.array(event_assoc)  # e.g. [[0,1,2], [2,0,1], [1,0,2], ...]

        # Combine features
        flat_feats = np.column_stack((
            event_bx, event_by, event_bz, event_raw_energy,
            event_beta, event_bphi,
            event_EV1, event_EV2, event_EV3,
            event_eV0x, event_eV0y, event_eV0z,
            event_sigma1, event_sigma2, event_sigma3
        ))
        x = torch.from_numpy(flat_feats).float()
        assoc = event_assoc

        total_tracksters = len(event_time)

        # --------------------------------------------------------------------
        # Group tracksters by their association tuple. Two tracksters belong
        # to the same pion group if their association arrays (converted to tuples)
        # match.
        # --------------------------------------------------------------------
        # Group tracksters by their association tuple
         # Group tracksters by the first element of event_assoc
        assoc_groups = {}
        for i, assoc in enumerate(event_assoc):
            key = assoc[0]  # Only use the first element as the key
            if key not in assoc_groups:
                assoc_groups[key] = []
            assoc_groups[key].append(i)
        #assoc_array = reconstruct_array(assoc_groups)
        pos_edges = []
        neg_edges = []
        # Ensure positive edges always connect to another trackster in the same group if possible
        for i in range(total_tracksters):
            key = event_assoc[i][0]  # Get first element as group identifier
            same_group = assoc_groups[key]

            # --- Positive edge ---
            if len(same_group) > 1:
                # Always select another trackster from the same group
                pos_target = random.choice([j for j in same_group if j != i])
            else:
                # No other trackster in the group, form a self-loop
                pos_target = i
            pos_edges.append([i, pos_target])

            # --- Negative edge ---
            neg_candidates = [j for j in range(total_tracksters) if event_assoc[j][0] != key]
            if neg_candidates:
                neg_target = random.choice(neg_candidates)
            else:
                neg_target = i
            neg_edges.append([i, neg_target])

        x_pos_edge = torch.tensor(pos_edges, dtype=torch.long)
        x_neg_edge = torch.tensor(neg_edges, dtype=torch.long)

        return Data(x=x, x_pe=x_pos_edge, x_ne=x_neg_edge, assoc = assoc)

In [31]:
ipath = "/vols/cms/mm1221/Data/100k/5pi/train/"

In [32]:
data_test = CCV1(ipath, max_events=1000, inp = 'train')

### Loading tracksters data


  0%|                                                                                                                                                             | 0/1 [00:00<?, ?it/s]

/vols/cms/mm1221/Data/100k/5pi/train/raw/train.root
80000
false 1
true 2
true 4
true 4
false 1
true 5
true 3
true 5
true 4
true 2
true 3
true 4
true 2
true 2
true 3
true 3
true 2
true 2
true 2
true 2
true 4
false 1
true 2
true 2
true 4
0 0
true 4
true 2
false 1
true 5
true 5
true 4
true 2
0 0
false 1
true 5
true 5
true 2
true 4
true 2
true 2
true 4
true 3
true 5
true 2
true 4
true 5
true 5
true 3
false 1
true 5
true 5
true 2
true 4
false 1
true 3
true 5
true 2
true 5
true 4
true 3
true 2
false 1
true 2
false 1
true 5
false 1
true 3
true 3
true 2
true 3
false 1
true 2
true 2
true 2
true 2
true 3
true 2
true 2
false 1
false 1
true 2
true 4
true 2
false 1
true 3
false 1
true 3
true 2
false 1
false 1
true 4
true 4
true 2
false 1
true 2
false 1
true 4
true 2
true 3
false 1
false 1
true 5
true 4
true 5
true 2
false 1
false 1
true 4
true 4
true 2
true 3
false 1
false 1
true 4
true 4
true 3
false 1
true 3
true 5
true 2
true 4
false 1
true 2
true 3
true 2
true 3
true 2
true 3
true 5
true 3
true

true 3
true 2
true 3
true 2
false 1
false 1
true 4
true 3
false 1
true 3
false 1
true 3
true 2
true 2
true 4
false 1
true 2
true 5
true 3
true 3
true 3
false 1
false 1
true 3
true 3
true 5
true 5
true 4
true 3
true 3
false 1
false 1
true 3
true 3
false 1
true 3
true 2
true 5
true 3
true 3
true 2
true 2
true 4
0 0
true 5
true 3
true 4
false 1
false 1
false 1
false 1
true 4
true 2
true 5
true 4
true 5
true 4
true 3
true 5
true 3
true 5
false 1
true 3
false 1
true 2
true 3
true 2
false 1
false 1
false 1
false 1
true 5
true 5
true 3
true 4
true 5
true 2
true 4
true 3
true 2
true 3
false 1
false 1
true 5
true 2
true 3
false 1
false 1
true 4
true 2
true 3
false 1
true 2
true 3
false 1
true 4
false 1
true 3
true 3
true 4
true 2
true 3
true 2
false 1
false 1
false 1
false 1
true 2
true 3
true 4
true 2
true 3
false 1
true 4
true 2
true 5
false 1
true 4
true 3
true 5
false 1
true 4
true 3
true 4
true 5
false 1
true 4
true 4
true 5
true 5
true 5
true 2
true 4
false 1
false 1
true 2
false 1
true 2

false 1
true 4
false 1
0 0
true 4
true 2
true 5
true 2
true 2
true 4
true 5
false 1
true 2
true 4
true 3
true 2
true 3
false 1
true 4
true 5
true 2
true 4
false 1
true 2
true 3
true 4
true 2
true 5
false 1
false 1
false 1
true 5
false 1
false 1
true 4
true 5
true 5
true 3
false 1
true 5
false 1
true 3
false 1
true 4
true 2
false 1
true 2
true 5
false 1
true 4
0 0
true 3
true 4
false 1
true 4
true 2
true 2
true 2
false 1
false 1
true 5
true 3
true 2
true 4
false 1
true 4
true 3
true 2
true 4
true 4
false 1
true 4
true 3
0 0
true 2
true 2
true 2
true 4
true 5
true 5
true 2
true 4
false 1
true 4
true 5
false 1
true 5
true 5
true 4
true 3
false 1
true 2
true 4
true 3
true 4
true 4
true 4
true 4
true 2
true 5
true 3
true 5
false 1
true 2
true 4
true 3
false 1
false 1
true 5
true 4
true 2
true 3
true 3
true 4
false 1
true 4
false 1
true 2
true 5
true 2
true 2
true 4
false 1
true 4
true 2
true 4
true 3
true 4
true 2
false 1
true 4
true 4
true 2
false 1
true 2
true 5
true 4
true 5
true 4
true 

...

false 8
false 19
false 30
false 21
false 28
false 18
false 18
false 17
false 18
false 17
false 12
false 6
false 23
false 29
false 13
false 30
false 24
false 13
false 11
false 43
false 14
false 10
false 24
true 3
false 25
false 19
false 15
false 28
false 20
true 4
false 14
true 4
false 16
false 7
true 4
false 21
false 16
false 9
false 21
false 23
false 18
false 20
false 26
true 5
false 28
false 30
false 25
false 11
false 21
false 34
false 7
false 18
false 9
false 6
false 20
false 28
false 41
false 22
false 10
false 10
false 17
false 20
false 22
false 26
false 18
false 6
true 3
false 31
false 19
false 26
false 10
false 17
false 31
false 15
false 15
false 30
false 34
false 26
false 15
false 20
false 6
false 13
false 41
false 16
false 16
false 20
false 10
false 11
false 13
false 26
false 18
false 24
false 6
false 22
false 31
false 22
false 7
false 21
false 23
false 32
false 33
false 27
false 13
false 24
false 6
false 21
false 8
false 17
false 25
false 30
false 16
false 28
false 21
0 0
fals

false 34
false 21
false 15
false 16
false 26
false 19
false 13
false 11
false 6
false 35
false 9
false 10
false 8
false 12
false 19
true 4
false 9
false 33
false 21
false 11
true 5
false 30
false 7
0 0
false 7
false 16
false 9
false 8
false 24
false 11
false 28
false 23
false 8
false 14
false 8
false 21
false 19
false 23
false 18
false 17
false 8
false 28
false 21
false 17
false 14
false 22
false 15
false 15
false 8
false 8
false 11
false 17
false 7
false 34
false 19
false 13
false 31
false 17
false 36
false 8
false 24
false 18
false 21
false 21
false 18
false 30
false 13
false 12
false 15
false 7
false 24
false 7
false 24
false 20
false 8
false 13
false 9
false 15
false 10
false 10
false 10
false 24
false 10
false 28
true 5
false 6
false 17
false 16
false 22
false 34
false 35
false 14
false 11
false 19
false 12
false 20
false 11
false 13
false 8
false 10
false 14
false 13
false 10
false 32
true 2
true 3
false 17
false 35
false 17
false 28
false 15
false 24
false 13
false 22
false 21
f

false 16
false 21
false 8
false 11
false 19
false 10
false 14
false 12
false 21
false 20
false 22
false 29
false 19
false 14
false 18
false 21
false 27
false 13
false 11
false 25
false 25
false 12
false 15
false 18
false 28
false 30
false 15
false 20
true 2
true 5
false 26
false 35
false 30
false 11
false 22
false 14
false 8
false 19
false 29
false 15
false 24
false 14
true 5
false 26
false 17
false 8
false 22
false 19
false 16
false 20
false 12
false 21
false 13
false 29
false 10
false 9
false 6
false 17
false 21
false 20
false 7
false 27
false 21
false 13
false 33
false 13
false 9
false 28
false 20
false 13
false 7
false 9
false 22
false 22
false 14
false 18
false 39
false 24
false 11
false 13
false 20
false 19
false 33
false 9
false 11
false 32
false 12
false 16
false 8
false 24
false 21
false 19
false 30
false 41
false 17
false 17
false 22
false 9
false 20
false 7
false 32
false 16
false 23
false 27
false 33
false 23
false 20
false 21
false 28
false 11
false 22
false 20
false 30
tr

  0%|                                                                                                                                                             | 0/1 [01:01<?, ?it/s]

 8
false 20
false 9
false 16
false 14
false 22
false 22
false 23
false 26
false 23
false 40
false 31
false 21
false 7
false 14
false 31
false 10
false 16
false 18
false 14
false 14
false 22
false 10
false 47
false 11
false 6
false 29
false 22
false 20
false 17
false 7
false 14
false 36
false 8
false 26
false 10
true 5
false 13
false 12
false 20
false 18
false 32
false 18
false 16
false 15
false 31
false 18
false 20
false 16
false 20
false 15
false 14
false 21
false 23
false 14
false 17
false 11
false 17
false 10
false 39
false 12
false 32
false 21
false 36
false 18
false 9
false 15
false 20
false 6
false 35
false 24
false 20
false 35
false 11
true 4
false 12
false 9
false 15
false 20
false 43
false 31
false 10
false 18
0 0
false 15
false 7
false 20
false 24
false 11
false 15
false 24
false 16
false 18
false 40
false 25
false 12
false 25
false 9
false 7
false 23
false 11
false 6
false 8
false 38
false 18
true 5
false 9
false 26
false 13
false 7
false 15
0 0
false 14
false 15
true 4
fals

In [33]:
print(data_test[30].x)

tensor([[-2.3257e+01,  7.1078e+01,  3.8901e+02,  1.5372e+01,  2.3512e+00,
          1.8870e+00,  1.0477e+02,  1.5146e+00,  1.0112e+00, -5.3932e-02,
          1.0218e-01,  9.9330e-01,  1.0228e+01,  1.1557e+00,  1.1595e+00],
        [-2.8275e+01,  7.7510e+01,  4.0889e+02,  2.7259e+00,  2.3038e+00,
          1.9206e+00,  2.8126e+02,  1.1414e+00,  1.0865e-01, -9.5034e-02,
          2.3271e-01,  9.6789e-01,  1.6728e+01,  1.0994e+00,  1.2106e+00],
        [-2.1110e+01,  6.2889e+01,  3.9448e+02,  1.6927e+00,  2.4830e+00,
          1.8946e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-2.3481e+01,  8.6262e+01,  4.2262e+02,  1.8014e+00,  2.2575e+00,
          1.8366e+00,  6.6652e+01,  3.1880e+00,  3.1757e-04, -1.1370e-02,
          3.2830e-01,  9.4451e-01,  8.1625e+00,  1.7739e+00,  2.5748e-01],
        [-2.3857e+01,  7.7507e+01,  4.0480e+02,  5.8917e+00,  2.3108e+00,
          1.8694e+00,  2.6400e+02,

In [34]:
import torch

# Assuming data_test is a list or similar iterable containing objects with an 'x' attribute
empty_count = 0
for item in data_test:
    if item.x.size(0) == 0:
        empty_count += 1

print(f"Number of empty 'x' tensors: {empty_count}")


Number of empty 'x' tensors: 0


In [11]:
v_file = uproot.open("/vols/cms/mm1221/Data/100k/5pi/train/raw/train.root")

In [12]:
bx = v_file['simtrackstersCP;22']['barycenter_x'].array()

In [21]:
ass = v_file['associations;4']['tsCLUE3D_simToReco_CP'].array()

In [23]:
print(len(bx[0]))
print(len(ass[0]))

1
1


In [24]:
skim_mask = []
for e in bx:
    if 2 <= len(e) <= 5:
        skim_mask.append(True)
    else:
        skim_mask.append(False)
    

In [25]:
bx = bx[skim_mask]
ass = ass[skim_mask]

In [29]:
print(bx[30])
print(ass[30])

[-14.7, -48]
[[], []]


In [229]:

test_loader = DataLoader(data_test, batch_size=64, shuffle=False, follow_batch=['x'])

/cvmfs/sft.cern.ch/lcg/views/LCG_105a_cuda/x86_64-el9-gcc11-opt/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [192]:
from model import Net
model = Net(64, 4, 0.1, 128)
checkpoint= torch.load('/vols/cms/mm1221/hgcal/pion5New/Track/StaticEdge/results/results_lr0.001_bs64_hd64_nl4_do0.1_k8_cd128/best_model.pt',  map_location=torch.device('cpu'))
model.load_state_dict(checkpoint)  
model.eval()  

Net(
  (lc_encode): Sequential(
    (0): Linear(in_features=15, out_features=64, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ELU(alpha=1.0)
  )
  (convs): ModuleList(
    (0-3): 4 x CustomStaticEdgeConv(
      (nn_module): Sequential(
        (0): Linear(in_features=128, out_features=64, bias=True)
        (1): ELU(alpha=1.0)
        (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (output): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): ELU(alpha=1.0)
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=32, out_features=16, bias=True)
    (4): ELU(alpha=1.0)
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=16, out_features=128, bias=True)
  )
)

In [193]:
#3: Make Predictions + Cluster -> Calculate the inference time
#3.1: Make Predictions
from torch_geometric.nn import knn_graph
all_predictions = []  

for i, data in enumerate(data_test):
    edge_index = knn_graph(data.x, k=8)  
    predictions = model(data.x, edge_index, 1)
    all_predictions.append(predictions[0].detach().cpu().numpy())  

all_predictions = np.array(all_predictions)

#3.2: Cluster using threshold found in Script A



/tmp/ipykernel_3622427/1522434933.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_predictions = np.array(all_predictions)


In [194]:
import numpy as np
import hdbscan

def HDBSCANClustering(all_predictions, 
                      min_cluster_size=5, 
                      min_samples=None, 
                      metric='euclidean', 
                      alpha=1.0,
                      cluster_selection_method='eom',
                      prediction_data=False,
                      allow_single_cluster=True,
                      core_dist_n_jobs=1,
                      cluster_selection_epsilon=0.0):
    """
    Performs HDBSCAN clustering on a list of prediction arrays with more hyperparameter control.

    Parameters:
    - all_predictions: List of numpy arrays, each containing data points for an event.
    - min_cluster_size: Minimum size of clusters.
    - min_samples: Number of samples in a neighborhood for a point to be considered a core point.
                   If None, it defaults to min_cluster_size.
    - metric: Distance metric to use.
    - alpha: Controls the balance between single linkage and average linkage clustering.
    - cluster_selection_method: 'eom' (Excess of Mass) or 'leaf' for finer clusters.
    - prediction_data: If True, allows later predictions on new data.
    - allow_single_cluster: If True, allows a single large cluster when applicable.
    - core_dist_n_jobs: Number of parallel jobs (-1 uses all cores).
    - cluster_selection_epsilon: Threshold distance for cluster selection (default 0.0).

    Returns:
    - all_cluster_labels: NumPy array of cluster labels for all events.
    """
    all_cluster_labels = []             

    for i, pred in enumerate(all_predictions):
        print(f"Processing event {i+1}/{len(all_predictions)}...")
        
        if len(pred) < 2:
            # Assign all points to cluster 0 (since HDBSCAN uses -1 for noise)
            cluster_labels = np.zeros(len(pred), dtype=int) 
        else:
            # Initialize HDBSCAN with specified parameters
            clusterer = hdbscan.HDBSCAN(
                min_cluster_size=min_cluster_size,
                min_samples=min_samples if min_samples is not None else min_cluster_size,
                metric=metric,
                alpha=alpha,
                cluster_selection_method=cluster_selection_method,
                prediction_data=prediction_data,
                allow_single_cluster=allow_single_cluster,
                core_dist_n_jobs=core_dist_n_jobs,
                cluster_selection_epsilon=cluster_selection_epsilon
            )
            
            # Perform clustering
            cluster_labels = clusterer.fit_predict(pred)  
        
        all_cluster_labels.append(cluster_labels)
    
    # Convert the list of cluster labels to a NumPy array
    all_cluster_labels = np.array(all_cluster_labels)
    return all_cluster_labels

all_cluster_labels = HDBSCANClustering(
    all_predictions, 
    min_cluster_size=2,  # Ensures at least 3 points per cluster
    metric='euclidean',  # Change distance metric
    alpha=1.0,  # Increase single linkage weighting
    cluster_selection_method='eom',  # Allow finer clusters
    prediction_data=False,  # Enable future prediction capability
    allow_single_cluster=False,  # Allow one large cluster
)


Processing event 1/1528...
Processing event 2/1528...
Processing event 3/1528...
Processing event 4/1528...
Processing event 5/1528...
Processing event 6/1528...
Processing event 7/1528...
Processing event 8/1528...
Processing event 9/1528...
Processing event 10/1528...
Processing event 11/1528...
Processing event 12/1528...
Processing event 13/1528...
Processing event 14/1528...
Processing event 15/1528...
Processing event 16/1528...
Processing event 17/1528...
Processing event 18/1528...
Processing event 19/1528...
Processing event 20/1528...
Processing event 21/1528...
Processing event 22/1528...
Processing event 23/1528...
Processing event 24/1528...
Processing event 25/1528...
Processing event 26/1528...
Processing event 27/1528...
Processing event 28/1528...
Processing event 29/1528...
Processing event 30/1528...
Processing event 31/1528...
Processing event 32/1528...
Processing event 33/1528...
Processing event 34/1528...
Processing event 35/1528...
Processing event 36/1528...
P

Processing event 314/1528...
Processing event 315/1528...
Processing event 316/1528...
Processing event 317/1528...
Processing event 318/1528...
Processing event 319/1528...
Processing event 320/1528...
Processing event 321/1528...
Processing event 322/1528...
Processing event 323/1528...
Processing event 324/1528...
Processing event 325/1528...
Processing event 326/1528...
Processing event 327/1528...
Processing event 328/1528...
Processing event 329/1528...
Processing event 330/1528...
Processing event 331/1528...
Processing event 332/1528...
Processing event 333/1528...
Processing event 334/1528...
Processing event 335/1528...
Processing event 336/1528...
Processing event 337/1528...
Processing event 338/1528...
Processing event 339/1528...
Processing event 340/1528...
Processing event 341/1528...
Processing event 342/1528...
Processing event 343/1528...
Processing event 344/1528...
Processing event 345/1528...
Processing event 346/1528...
Processing event 347/1528...
Processing eve

Processing event 630/1528...
Processing event 631/1528...
Processing event 632/1528...
Processing event 633/1528...
Processing event 634/1528...
Processing event 635/1528...
Processing event 636/1528...
Processing event 637/1528...
Processing event 638/1528...
Processing event 639/1528...
Processing event 640/1528...
Processing event 641/1528...
Processing event 642/1528...
Processing event 643/1528...
Processing event 644/1528...
Processing event 645/1528...
Processing event 646/1528...
Processing event 647/1528...
Processing event 648/1528...
Processing event 649/1528...
Processing event 650/1528...
Processing event 651/1528...
Processing event 652/1528...
Processing event 653/1528...
Processing event 654/1528...
Processing event 655/1528...
Processing event 656/1528...
Processing event 657/1528...
Processing event 658/1528...
Processing event 659/1528...
Processing event 660/1528...
Processing event 661/1528...
Processing event 662/1528...
Processing event 663/1528...
Processing eve

Processing event 932/1528...
Processing event 933/1528...
Processing event 934/1528...
Processing event 935/1528...
Processing event 936/1528...
Processing event 937/1528...
Processing event 938/1528...
Processing event 939/1528...
Processing event 940/1528...
Processing event 941/1528...
Processing event 942/1528...
Processing event 943/1528...
Processing event 944/1528...
Processing event 945/1528...
Processing event 946/1528...
Processing event 947/1528...
Processing event 948/1528...
Processing event 949/1528...
Processing event 950/1528...
Processing event 951/1528...
Processing event 952/1528...
Processing event 953/1528...
Processing event 954/1528...
Processing event 955/1528...
Processing event 956/1528...
Processing event 957/1528...
Processing event 958/1528...
Processing event 959/1528...
Processing event 960/1528...
Processing event 961/1528...
Processing event 962/1528...
Processing event 963/1528...
Processing event 964/1528...
Processing event 965/1528...
Processing eve

Processing event 1241/1528...
Processing event 1242/1528...
Processing event 1243/1528...
Processing event 1244/1528...
Processing event 1245/1528...
Processing event 1246/1528...
Processing event 1247/1528...
Processing event 1248/1528...
Processing event 1249/1528...
Processing event 1250/1528...
Processing event 1251/1528...
Processing event 1252/1528...
Processing event 1253/1528...
Processing event 1254/1528...
Processing event 1255/1528...
Processing event 1256/1528...
Processing event 1257/1528...
Processing event 1258/1528...
Processing event 1259/1528...
Processing event 1260/1528...
Processing event 1261/1528...
Processing event 1262/1528...
Processing event 1263/1528...
Processing event 1264/1528...
Processing event 1265/1528...
Processing event 1266/1528...
Processing event 1267/1528...
Processing event 1268/1528...
Processing event 1269/1528...
Processing event 1270/1528...
Processing event 1271/1528...
Processing event 1272/1528...
Processing event 1273/1528...
Processing

/tmp/ipykernel_3622427/2414487783.py:61: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_cluster_labels = np.array(all_cluster_labels)


In [208]:
print(all_cluster_labels[0])

[0 0 0 0 0 0 0 0 0 1 1 1 0 1]


In [195]:
from collections import defaultdict

# Dictionary to store indices for each unique value per event
recon_ind = []

# Loop over all events
for event in all_cluster_labels:
    grouped_indices = defaultdict(list)
    
    # Populate the dictionary for the current event
    for idx, value in enumerate(event):
        grouped_indices[value].append(idx)
    
    # Convert to a regular dictionary and store in the list
    recon_ind.append(dict(grouped_indices))


In [198]:
print(recon_ind[1])

{-1: [0, 1, 2, 3, 4]}


In [199]:
print(data_test[0].x)

tensor([[ 4.0012e+01, -5.0409e+01,  3.3059e+02,  1.0697e+00,  2.3389e+00,
         -8.9988e-01,  1.8502e+01,  9.2510e-01,  2.2303e-02,  2.9578e-01,
          2.2548e-01,  9.2826e-01,  3.4947e+00,  2.5602e+00,  8.2572e-01],
        [ 3.9335e+01, -5.4234e+01,  3.3929e+02,  7.6161e+01,  2.3249e+00,
         -9.4330e-01,  8.4689e+01,  1.4449e+00,  1.1959e+00,  1.4545e-01,
         -1.8238e-01,  9.7241e-01,  8.8217e+00,  1.1800e+00,  2.8487e+00],
        [ 4.2992e+01, -5.3728e+01,  3.3630e+02,  1.3437e+00,  2.2901e+00,
         -8.9594e-01,  9.4715e+00,  2.2211e+00,  7.3053e-01, -3.2082e-01,
         -5.2148e-02,  9.4570e-01,  2.7115e+00,  1.7891e+00,  1.3675e+00],
        [ 4.7899e+01, -5.2529e+01,  3.3855e+02,  2.1946e+00,  2.2647e+00,
         -8.3147e-01,  7.4109e+00,  2.1598e-01,  7.4941e-15,  7.3012e-01,
         -1.0384e-01,  6.7538e-01,  1.3029e+00,  1.8352e+00,  1.6004e+00],
        [ 3.8504e+01, -5.0554e+01,  3.4295e+02,  1.5969e+00,  2.3874e+00,
         -9.1989e-01,  1.4286e+01,

In [236]:
import torch
from collections import defaultdict

def calculate_energy_diff_ratio(recon_ind, assoc_array, data_x):
    """
    Calculate the total energy difference ratio across all calo particles in an event.

    Args:
        recon_ind (dict): Mapping of cluster IDs to lists of indices.
                           Example: {0: [0,1,2,...], 1: [9,10,11,...]}
        assoc_array (list or numpy array): Array where each element represents the calo particle ID for the corresponding index.
                                           Example: [2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0]
        data_x (torch.Tensor): Tensor of shape [num_indices, 15], with energy at index 3.

    Returns:
        float: Total energy difference ratio for the event.
    """
    total_energy_diff_ratio = 0.0

    energies = data_x[:, 3].cpu()  # Extract energy column

    # Identify unique calo particle IDs
    unique_calo_ids = set(assoc_array)

    for calo_id in unique_calo_ids:
        # Get all indices belonging to this calo particle
        calo_indices = [idx for idx, cid in enumerate(assoc_array) if cid == calo_id]

        if not calo_indices:
            continue  # Skip if no indices found

        # Calculate total true energy for the calo particle
        calo_energy = energies[calo_indices].sum().item()

        best_shared_energy = 0.0

        # Iterate over all reconstructed tracksters to find the best match
        for trackster_indices in recon_ind.values():
            # Find shared indices between calo particle and trackster
            shared_indices = set(calo_indices).intersection(trackster_indices)

            if shared_indices:
                # Sum energies of shared indices
                shared_energy = energies[list(shared_indices)].sum().item()

                # Update best_shared_energy if this trackster has higher shared energy
                if shared_energy > best_shared_energy:
                    best_shared_energy = shared_energy

        # Compute energy difference ratio
        if calo_energy > 0:
            energy_diff_ratio = (calo_energy - best_shared_energy) / calo_energy
        else:
            energy_diff_ratio = 0.0  # Avoid division by zero

        # Accumulate the ratio
        total_energy_diff_ratio += energy_diff_ratio

    return total_energy_diff_ratio


In [239]:
calculate_energy_diff_ratio(recon_ind[0], data_test[0].assoc, data_test[0].x)

0.23705877650943383